In [7]:
#%% Imports:
import matplotlib
matplotlib.use('MACOSX')

from math import sqrt
import os
import numpy as np
import mne
#import matplotlib #doesnt it import through mne already?
from copy import deepcopy

#Open data:
#sample_data_folder = mne.datasets.sample.data_path()
#kath_raw_file2 = "/Users/jenya/Documents/Oldenburg and university/Job Uni Rieger lab/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif"
#kath_raw_file = os.path.join('Katharinas_Data','sub_HT05ND16', '210811', 'mikado-1.fif')
#kath_raw_file='/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif'
kath_raw_file='./data/sub_HT05ND16/210811/mikado-1.fif/'
print(kath_raw_file)
#print(kath_raw_file2)                                   
raw = mne.io.read_raw_fif(kath_raw_file)
#raw.crop(0, 60).load_data()  # just use a fraction of data for speed here

#Print info about the data:
print(raw)
print(raw.info)
raw

#crop the data to calculate faster

raw_cropped = raw.copy()
raw_cropped.crop(0, 300) #(first 5 min)

./data/sub_HT05ND16/210811/mikado-1.fif/
Opening raw data file ./data/sub_HT05ND16/210811/mikado-1.fif/...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v4 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v5 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v6 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v7 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v8 (1 x 306)  idle
    Range : 1809000 ... 3375999 =   1809.000 ...  3375.999 secs
Ready.
Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/sub_HT05ND16/210811/mikado-2.fif...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
     

/var/folders/h2/khhmb4p510vg63hbv0qkftt80000gs/T/ipykernel_18359/1876021669.py:20: RuntimeWarning: This filename (./data/sub_HT05ND16/210811/mikado-1.fif/) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(kath_raw_file)


Ready.
<Raw | mikado-1.fif, 319 x 3390000 (3390.0 s), ~6.6 MB, data not loaded>
<Info | 24 non-empty values
 acq_pars: ACQactiveGround 1 ACQch.BIO001.gain 2000 ACQch.BIO001.highpass ...
 bads: []
 ch_names: IASX+, IASX-, IASY+, IASY-, IASZ+, IASZ-, IAS_DX, IAS_DY, IAS_X, ...
 chs: 11 IAS, 102 Magnetometers, 204 Gradiometers, 1 Stimulus, 1 SYST
 custom_ref_applied: False
 description: TRIUX system at Oldenburg_3127
 dev_head_t: MEG device -> head transform
 dig: 356 items (3 Cardinal, 5 HPI, 348 Extra)
 events: 1 item (list)
 experimenter: Meg User (meg)
 file_id: 4 items (dict)
 gantry_angle: 68.0
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 hpi_subsystem: 3 items (dict)
 line_freq: 50.0
 lowpass: 330.0 Hz
 meas_date: 2021-08-11 12:05:17 UTC
 meas_id: 4 items (dict)
 nchan: 319
 proj_id: 1 item (ndarray)
 proj_name: mikado
 projs: magn8_iasoff_68deg.fif : PCA-v1: off, magn8_iasoff_68deg.fif : ...
 sfreq: 1000.0 Hz
 subject_info: 6 items (dict)
>


<Raw | mikado-1.fif, 319 x 300001 (300.0 s), ~6.6 MB, data not loaded>

In [8]:
#%% Find magnetometers and gradiometers:
# unit t - magenetometer. unit M_T - gradiometer. (in this set name will end with 1 for magnet, 
# with 2and3 for grad.)

mags = []
grads = []

for i, chs in enumerate(raw_cropped.info['chs']):

    if str(chs['unit']).endswith('UNIT_T)'):
        mags.append((chs['ch_name'], i))
    elif str(chs['unit']).endswith('UNIT_T_M)'):
        grads.append((chs['ch_name'], i))

In [9]:
#epoch channels together

events = mne.find_events(raw_cropped, stim_channel='STI101', min_duration=1.2)
epochs = mne.Epochs(raw_cropped, events, tmin=-0.2, tmax=1, preload=True, baseline = None) #, event_id=event_dict, reject=reject_criteria)
#df = epochs.to_data_frame(time_format=None, scalings=dict(mag=1, grad=1))
df = epochs.to_data_frame(time_format=None, scalings=None)

#by default, channel measurement values are scaled so that EEG data are converted 
# to µV, magnetometer data are converted to fT, and gradiometer data are converted 
# to fT/cm. These scalings can be customized through the scalings parameter, or 
# suppressed by passing scalings=dict(eeg=1, mag=1, grad=1).
df



Trigger channel has a non-zero initial value of 18 (consider using initial_event=True to detect this event)
8 events found
Event IDs: [ 9 19 20 21 22]
Not setting metadata
Not setting metadata
8 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 8)
8 projection items activated
Loading data for 8 events and 1201 original time points ...
0 bad epochs dropped


/var/folders/h2/khhmb4p510vg63hbv0qkftt80000gs/T/ipykernel_18359/740392597.py:3: RuntimeWarning: Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug
  events = mne.find_events(raw_cropped, stim_channel='STI101', min_duration=1.2)


,time,condition,epoch,IASX+,IASX-,IASY+,IASY-,IASZ+,IASZ-,IAS_DX,...,MEG2622,MEG2623,MEG2631,MEG2632,MEG2633,MEG2641,MEG2642,MEG2643,STI101,SYS201
0,-0.200,19,0,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.095108e-29,-5.180752e-29,-1.192093e-06,...,11.947632,77.659607,-154.700281,-93.589782,-73.677063,-72.181185,-137.397766,65.711975,18.0,0.0
1,-0.199,19,0,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,...,101.554870,-55.755615,-84.452583,-47.790527,-65.711975,48.780844,-55.755615,89.607239,18.0,0.0
2,-0.198,19,0,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,-1.192093e-06,...,55.755615,-153.327942,-16.318079,-83.633423,-137.397766,143.571248,-37.834167,77.659607,18.0,0.0
3,-0.197,19,0,7.974282e-29,-1.192093e-06,-9.600046e-31,2.019014e-29,-1.192093e-06,-1.192093e-06,7.317702e-30,...,19.912720,19.912720,-93.718898,-109.519958,-129.432678,2.456428,-77.659607,5.973816,18.0,0.0
4,-0.196,19,0,7.337872e-29,7.717356e-29,5.197823e-30,1.919949e-29,-1.581347e-29,-7.481316e-29,8.938612e-30,...,91.598511,141.380310,-169.302916,-15.930176,-1.991272,-166.838272,19.912720,49.781799,18.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9603,0.996,20,7,-1.192093e-06,-1.192093e-06,-1.683182e-29,6.909267e-31,1.010209e-30,1.681756e-29,-1.192093e-06,...,75.668335,5.973816,-182.567912,55.755615,65.711975,42.247881,-59.738159,99.563598,20.0,0.0
9604,0.997,20,7,-1.192093e-06,4.738726e-30,-1.192093e-06,-1.192093e-06,-1.358453e-30,-1.885637e-30,-1.192093e-06,...,133.415222,39.825439,-148.620201,61.729431,0.000000,-21.741849,-77.659607,9.956360,20.0,0.0
9605,0.998,20,7,-1.192093e-06,-1.726060e-29,-1.192093e-06,-1.192093e-06,9.504517e-30,3.352403e-29,-1.192093e-06,...,97.572326,-17.921448,-83.675775,-1.991272,-11.947632,-3.944463,-143.371582,-37.834167,20.0,0.0
9606,0.999,20,7,9.138198e-30,-1.909005e-29,-1.192093e-06,-1.192093e-06,1.661232e-29,5.756178e-29,-1.192093e-06,...,97.572326,-25.886536,-188.398138,-71.685791,-19.912720,51.929438,-201.118469,-3.982544,20.0,0.0


In [10]:
#epoch channels separately

picks_grad = mne.pick_types(raw_cropped.info, meg='grad', eeg=False, eog=False, stim=False)
picks_magn = mne.pick_types(raw_cropped.info, meg='mag', eeg=False, eog=False, stim=False)

epochs_mags = mne.Epochs(raw_cropped, events, picks=picks_magn, tmin=-0.2, tmax=1, preload=True, baseline = None)
epochs_grads = mne.Epochs(raw_cropped, events, picks=picks_grad, tmin=-0.2, tmax=1, preload=True, baseline = None)

#Present epochs as data frame - separately for mags and grads

#df_epochs_mags = epochs_mags.to_data_frame(time_format=None, scalings=dict(mag=1, grad=1))
df_epochs_mags = epochs_mags.to_data_frame(time_format=None, scalings=None)

#df_epochs_grads = epochs_grads.to_data_frame(time_format=None, scalings=dict(mag=1, grad=1))
df_epochs_grads = epochs_grads.to_data_frame(time_format=None, scalings=None)

df_epochs_mags

Not setting metadata
Not setting metadata
8 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 8)
8 projection items activated
Loading data for 8 events and 1201 original time points ...
0 bad epochs dropped
Not setting metadata
Not setting metadata
8 matching events found
No baseline correction applied
8 projection items activated
Loading data for 8 events and 1201 original time points ...
0 bad epochs dropped


,time,condition,epoch,MEG0111,MEG0121,MEG0131,MEG0141,MEG0211,MEG0221,MEG0231,...,MEG2431,MEG2441,MEG2511,MEG2521,MEG2531,MEG2541,MEG2611,MEG2621,MEG2631,MEG2641
0,-0.200,19,0,-105.254999,-101.028080,-185.490119,-20.463350,-87.336121,-40.089553,1.126569,...,181.178745,104.046122,123.254512,27.499431,67.413738,-131.676764,113.871037,-105.041274,-154.700281,-72.181185
1,-0.199,19,0,-71.763932,-69.898172,-132.205426,-123.543450,-122.153991,31.857612,2.634028,...,173.709063,134.629104,234.205768,45.460081,103.007664,-0.510377,132.809294,-101.566970,-84.452583,48.780844
2,-0.198,19,0,-59.819084,-61.886269,-118.677611,-50.275990,-62.810864,116.318292,-31.002455,...,91.768120,122.307415,208.950812,28.543160,110.302033,134.177279,143.085353,-143.851285,-16.318079,143.571248
3,-0.197,19,0,-25.975807,-2.902860,-178.301643,-76.866040,41.329409,64.150450,53.149750,...,55.815054,45.160739,116.492060,-92.174386,-2.699964,98.654525,125.727134,-96.631968,-93.718898,2.456428
4,-0.196,19,0,11.490839,-6.599419,-153.914943,-48.780480,31.455754,-19.860444,45.288277,...,64.432246,55.144807,138.302190,-258.804802,-44.379998,6.780823,18.620744,-7.515577,-169.302916,-166.838272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9603,0.996,20,7,-62.287145,122.084463,-63.815965,16.364839,84.785140,86.490182,30.577488,...,-31.572502,13.886193,46.513789,-186.922433,-102.015770,54.298327,113.535757,-242.489962,-182.567912,42.247881
9604,0.997,20,7,39.199763,50.973766,-242.985989,-61.979318,-53.291201,69.421759,-122.612740,...,49.600028,75.473810,22.731092,-145.270249,-183.812570,-51.084896,192.560132,-187.268789,-148.620201,-21.741849
9605,0.998,20,7,-33.047691,-192.844831,-92.401154,-55.162891,47.162830,55.311639,-92.741508,...,89.431914,84.016627,-128.729505,-73.500859,-200.914654,-261.360472,165.540804,-41.223239,-83.675775,-3.944463
9606,0.999,20,7,-89.461351,-268.774507,59.248641,9.514944,45.578212,69.351534,141.635076,...,45.364928,134.208556,-77.981183,-16.675615,-154.729144,-252.188627,80.196217,-137.092445,-188.398138,51.929438


In [11]:
#Compare data in 2 dfs: mags and grads together vs mags separately

#Joined df:
epoch0_ind=df.index[df['epoch'] == 0].tolist()
ch0111=df.iloc[epoch0_ind, 3+11] #all data of epoch 0 for MEG0111. 
#index of this channel in raw: 11. moved by 3 elements because data frame creates 3 additional columns
#print('data for MEG0111 of epoch 0: ', ch0111)

#Separate mags df:
epoch0_ind_mag=df_epochs_mags.index[df_epochs_mags['epoch'] == 0].tolist()
ch0111_mag=df_epochs_mags.iloc[epoch0_ind, 3] #all data of epoch 0 for MEG0111. 
#index only moved by 3 elements here only, since no extra channels in this df

#print('dfs have same data: ')
#print("A: ", ch0111[0], "\nB: ",ch0111_mag[0]) #check if the data in joined df and the magnetometer df for channel MEG0111 is same in both)

for index, value in enumerate(ch0111):
    if value != ch0111_mag[index]:
        print("Not equal:", index, "\n", value, "\n", ch0111_mag[index], "\n")

Not equal: 0 
 -105.25499934360496 
 -105.25499935987298 

Not equal: 1 
 -71.763931661001 
 -71.76393163677511 

Not equal: 2 
 -59.819084314754015 
 -59.81908429052783 

Not equal: 3 
 -25.97580725884481 
 -25.97580722066308 

Not equal: 4 
 11.49083903000234 
 11.490839030001592 

Not equal: 5 
 -33.315197160088864 
 -33.31519716008956 

Not equal: 6 
 -29.402017982994508 
 -29.402017982995456 

Not equal: 7 
 -39.89772798021136 
 -39.89772798467011 

Not equal: 8 
 -85.69238240724178 
 -85.69238241401159 

Not equal: 9 
 15.372183650848623 
 15.372183644078945 

Not equal: 10 
 43.76882857957955 
 43.768828563312006 

Not equal: 11 
 -38.37030649701544 
 -38.37030647278976 

Not equal: 12 
 -45.3018629180047 
 -45.30186289365872 

Not equal: 13 
 22.52575580004412 
 22.525755831038754 

Not equal: 14 
 -20.870677025482934 
 -20.87067702548428 

Not equal: 15 
 -149.16048693097798 
 -149.16048693097943 

Not equal: 16 
 -94.4264672287222 
 -94.42646722884416 

Not equal: 17 
 -0.747

In [12]:
#Data of f this channel and epoch from  joined DF:
print(df.iloc[epoch0_ind, 3+11])

0      -105.254999
1       -71.763932
2       -59.819084
3       -25.975807
4        11.490839
           ...    
1196    -15.705636
1197     86.770152
1198     28.540451
1199   -100.388320
1200     46.652600
Name: MEG0111, Length: 1201, dtype: float64


In [13]:
#Data of this channel and epoch from separated DF:
df_epochs_mags.iloc[epoch0_ind, 3]

0      -105.254999
1       -71.763932
2       -59.819084
3       -25.975807
4        11.490839
           ...    
1196    -15.705636
1197     86.770152
1198     28.540451
1199   -100.388320
1200     46.652600
Name: MEG0111, Length: 1201, dtype: float64